<a href="https://colab.research.google.com/github/haizznaam/Content-Based-and-Tensorflow-Movie-Recommender-System/blob/main/knn_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.chdir('/content/drive/MyDrive/Movie Recommendation System/The-Movie-Dataset/')
print("We are currently in the folder of ", os.getcwd())

We are currently in the folder of  /content/drive/MyDrive/Movie Recommendation System/The-Movie-Dataset


# [KNN](#KNN)
- ## [Data preperation](##KNN-Data-preperation)
- ## [KNN Implamentation and Training](##KNN-Implamentation-and-Training)
- ## [Implamenting Prediction and Evaluation](##Implamenting-Prediction-and-Evaluation)
- ## [Testing different K values](##Testing-different-K-values)
- ## [Adding new Users or Items or rating and adressing the cold start problem](##Adding-new-Users-or-Items-or-rating-and-adressing-the-cold-start-problem)

# KNN

Understanding KNN:

https://www.youtube.com/watch?v=jw5LhTWUoG4&t=127s

https://www.youtube.com/watch?v=kccT0FVK6OY

https://chat.openai.com/share/837847bb-ae5f-4cc3-a8a3-a664f352365f

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

Prepearing the data for KNN is a bit different then for SVD. We are gonna look at two examples 1. Where we have all possible user entries in our KNN matrix and all of our possible item entries. So in other words we have all the users and movies that are ever going to be used on our algorithem.

In the real world things area bit different since new users and moveis / items are added every day. the 2. metod we will look at is upgrading our algorithem to be able to handle  new users and items respectively.  

## KNN Data preperation

## 1. Static user and item entries no new users
For this aproach we will gaurantee that each user and each item has an entry in our trainign dataset. Because for KNN we need entries to predict unlike SVD. That's because KNN works on other user's rating for similar entries unlike SVD that mathamaticaly models latent factors and can "implicitly" "learn" what similar data might be rated as.

Đối với phương pháp này, chúng tôi sẽ đảm bảo rằng mỗi user và mỗi item đều có một entry trong tập dữ liệu huấn luyện của chúng tôi.

Vì đối với KNN chúng ta cần các entries để dự đoán,  không giống như SVD. Đó là bởi vì KNN hoạt động dựa trên xếp hạng của người dùng khác cho các mục tương tự không giống như SVD mà toán học mô hình hóa các yếu tố tiềm ẩn và có thể "ngầm" "tìm hiểu" dữ liệu tương tự có thể được xếp hạng là gì.

The following is my first intuitive attempt at spliting and prepearing the data, but it is wrong we will get to that shortly:

In [5]:
# Load the dataset
df = pd.read_csv('ratings_small.csv')

# Display the first few rows of the dataframe
df.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']
df.head()

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
# Perform a train-test split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Check the size of the resulting splits
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")

Train data size: (80003, 4)
Test data size: (20001, 4)


In [8]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of unique users:", num_users)
print("number of unique items:", num_items)

number of unique users: 671
number of unique items: 8399


In [9]:
# Find the minimum user_id in the train and test datasets
min_user_id_train = train_data['user_id'].min()
max_user_id_train = train_data['user_id'].max()

print(f"Minimum user_id in train dataset: {min_user_id_train}")
print(f"Maximum user_id in train dataset: {max_user_id_train}")

print('=' * 40)
# Find the maximum user_id in the train and test datasets
min_user_id_test = test_data['user_id'].min()
max_user_id_test = test_data['user_id'].max()

print(f"Minimum user_id in test dataset: {min_user_id_test}")
print(f"Maximum user_id in test dataset: {max_user_id_test}")

Minimum user_id in train dataset: 1
Maximum user_id in train dataset: 671
Minimum user_id in test dataset: 1
Maximum user_id in test dataset: 671


In [10]:
# Find the minimum item_id in the train and test datasets
min_item_id_train = train_data['item_id'].min()
max_item_id_train = train_data['item_id'].max()

print(f"Minimum item_id in train dataset: {min_item_id_train}")
print(f"Maximum item_id in train dataset: {max_item_id_train}")

print('=' * 40)
# Find the maximum item_id in the train and test datasets
min_item_id_test = test_data['item_id'].min()
max_item_id_test = test_data['item_id'].max()

print(f"Minimum item_id in test dataset: {min_item_id_test}")
print(f"Maximum item_id in test dataset: {max_item_id_test}")

Minimum item_id in train dataset: 1
Maximum item_id in train dataset: 163949
Minimum item_id in test dataset: 1
Maximum item_id in test dataset: 162672


In [11]:
last_user = test_data['user_id'].max()
last_item = test_data['item_id'].max()
print("last user id:", last_user)
print("last item id:", last_item)

last user id: 671
last item id: 162672


In [12]:
train_data.head()

,user_id,item_id,rating,timestamp
37865,273,5816,4.5,1466946328
46342,339,2028,4.5,1446663181
64614,461,3895,0.5,1093224965
41974,300,3578,4.5,1086010878
50236,369,292,3.0,847465462


In [13]:
# Create a pivot table for the train data to form the user-item matrix
train_data_matrix_0_fill = train_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Display the shape and first few rows of the matrix
display(train_data_matrix_0_fill.head())
train_data_matrix_0_fill = train_data_matrix_0_fill.to_numpy()
print(train_data_matrix_0_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,160656,160718,161084,161155,161594,161830,161918,161944,162542,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0,0.0,0.0,0,0,0.0,0,0,0


(671, 8399)


The reason that the above code for prepearing data is wrong is because in the above code I didn't implicitly make sure that our train_data_matrix_0_fill has an entry for each possible user and each possible item in our dataset.
As we can see:

In [14]:
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()
print("number of full unique users:", num_users)
print("number of full unique items:", num_items)

number of full unique users: 671
number of full unique items: 9066


In [15]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of train unique users:", num_users)
print("number of train unique items:", num_items)

number of train unique users: 671
number of train unique items: 8399


As we can see from the above codse in our training data that we split we have 671 users which is ok but we only have 8399 items which is not okey because when we look at the entire dataset(the df dataset) without the split we can see that we have 9066 movies and 8399 != 9066...

Also if we look at the biggest values of user id's and item id's in our test data we quickly see the same problem:

In [16]:
last_user = train_data['user_id'].max()
last_item = train_data['item_id'].max()
print("last user id:", last_user)
print("last item id:", last_item)

print("!!!does not equal to:")
print("number of unique users:", num_users)
print("number of unique items:", num_items)

last user id: 671
last item id: 163949
!!!does not equal to:
number of unique users: 671
number of unique items: 8399


We can again see that in our test data we have a item with the biggest index id of 163949 but we only have 8399 items in our training dataset because the random split didn't make sure all of our possible values are represneted in out training set and that is a problem for KNN unless we update all the values in our Knn matrix which we will explore in a future step. For now let's fix this problem by making sure each user, item id value is represented in our training data:

SO as we just astablished the previous data preperation and split was wrong and so we will rather do it like:

Here is what we will do in the code:

- take the df dataset and duplicate it into df_copy that isn't split yet.

- Then firstly find the first apearence of each unique user id and take the entry in df_copy out of df_copy and add it to training_data.

- Then find the first unique apearence of each item id in df (not df_copy) if it already exists inside of train_data and dosen't inside of df_copy (so it was taken out of df copy allready when we were looking for apearences of user id's ) then don't do anything.

- If on the other hand it isn't inside of train_data and it is inside fo df_copy again take it out of df_copy and put it into train_data.

- After that split the df_copy like we did with df before a 75 / 25 split into training_remaining and test_data. Then simply add the training_remeining onto training_data.

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming df is already loaded
df_copy = df.copy()

# Collect indices for users
user_indices = []
for user_id in df['user_id'].unique():
    user_index = df_copy[df_copy['user_id'] == user_id].index[0]
    user_indices.append(user_index)

# Collect indices for items not already covered by user selection
item_indices = []
for item_id in df['item_id'].unique():
    if not df_copy.loc[df_copy['item_id'] == item_id].index.intersection(user_indices).any():
        item_index = df_copy[df_copy['item_id'] == item_id].index[0]
        item_indices.append(item_index)

# Combine indices and drop duplicates in case of overlap
combined_indices = list(set(user_indices + item_indices))

# Extract rows for training data
train_data_initial = df_copy.loc[combined_indices]

# Drop these rows from df_copy
df_copy = df_copy.drop(index=combined_indices)

# Perform a 75/25 split on the remaining data
training_remaining, test_data = train_test_split(df_copy, test_size=0.25, random_state=42)

# Combine training_remaining with the initial extracted training data
train_data = pd.concat([train_data_initial, training_remaining]).reset_index(drop=True)

# Output the sizes of the final splits
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")


Train data size: (77411, 4)
Test data size: (22593, 4)


In [18]:
num_users = train_data['user_id'].nunique()
num_items = train_data['item_id'].nunique()
print("number of unique users:", num_users)
print("number of unique items:", num_items)

number of unique users: 671
number of unique items: 9066


In [19]:
last_user = test_data['user_id'].max()
last_item = test_data['item_id'].max()
print("test last user id:", last_user)
print("test last item id:", last_item)
print('=' * 40)
last_user = df['user_id'].max()
last_item = df['item_id'].max()
print("full last user id:", last_user)
print("full last item id:", last_item)

print('=' * 40)
last_user = train_data['user_id'].max()
last_item = train_data['item_id'].max()
print("train last user id:", last_user)
print("train last item id:", last_item)

test last user id: 671
test last item id: 159093
full last user id: 671
full last item id: 163949
train last user id: 671
train last item id: 163949


As we can see the values for df and train are now the same and that is important! The test sets last item id: 159093 is not the same but that is okey because we are not training on the test data so there is no problem in that regard.

In [20]:
# Create a pivot table for the train data to form the user-item matrix
train_data_matrix_0_fill = train_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)

# Display the shape and first few rows of the matrix
display(train_data_matrix_0_fill.head())
train_data_matrix_0_fill = train_data_matrix_0_fill.to_numpy()
print(train_data_matrix_0_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0,0,0.0,...,0.0,0.0,0,0,0.0,0,0.0,0,0,0


(671, 9066)


We can also do the same but use avrages instead of 0's to fill the missing values. We will use this initialization alter on to see if it makes any difference for now we will just prepare the matrix!

In [21]:
user_rating_avg = train_data.groupby('user_id')['rating'].mean()
user_rating_avg

user_id
1      2.550000
2      3.493333
3      3.590000
4      4.348259
5      3.917526
         ...   
667    3.666667
668    3.785714
669    3.444444
670    3.954545
671    3.925676
Name: rating, Length: 671, dtype: float64

In [22]:
# Create the pivot table
train_data_matrix = train_data.pivot_table(index='user_id', columns='item_id', values='rating')
# Fill missing values with user averages
train_data_matrix_avg_fill = train_data_matrix.apply(lambda x: x.fillna(user_rating_avg[x.name]), axis=1)
# Display the shape and first few rows of the matrix
train_data_matrix_avg_fill.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,...,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000
2,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,4.000000,...,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333
3,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,...,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000
4,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.000000,...,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259
5,3.917526,3.917526,4.000000,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,...,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526


## KNN Implamentation and Training

The following function calculates the similareties between two user entires based on their movie ratings. In this notebook we will mainly focus on the euclidian ecimilarity but later we will experiment with the cosine and paerson similareties

In [23]:
import numpy as np

def calculate_similarity_between_users(user1, user2, similarity_type='euclidean'):
    if similarity_type == 'euclidean':
        # Calculate the Euclidean distance
        distance = np.sqrt(np.nansum((user1 - user2) ** 2))
        # Convert distance to similarity
        similarity = 1 / (1 + distance)
        return similarity
    else:
        raise ValueError("Unsupported similarity type")

Below is the function that trains out KNN matrix. It find the k nearest neighbours based on the similareties between them. It also returns a similareties matrix that we can use for more fine tuned predictions

In [24]:
import time
def k_nearest_neighbors(dataset_matrix, k, similarity_type='euclidean', print_=True, print_every_n=100):
    # Convert the pandas DataFrame to a NumPy array if it's not already an array
    if isinstance(dataset_matrix, pd.DataFrame):
        dataset_array = dataset_matrix.to_numpy()
    else:
        dataset_array = dataset_matrix

    num_users = dataset_array.shape[0]
    # Initialize the KNN matrix with zeros
    knn_matrix = np.zeros((num_users, k), dtype=int)
    similarities_matrix = np.zeros((num_users, k), dtype=float)

    # Placeholder for the similarity scores and user indices
    # We use np.inf as the initial values to ensure any actual similarity score will be lower
    best_similarities = np.full((k, 2), np.inf) # [similarity, index]

    start_total_time = time.time()  # Start timing for the whole function

    for i in range(num_users):
        start_time = time.time()  # Start timing for the current user
        for j in range(num_users):
            if i != j:
                similarity = calculate_similarity_between_users(dataset_array[i], dataset_array[j], similarity_type)
                # Check if this similarity is better than the current stored similarities
                for l in range(k-1, -1, -1):
                    if similarity < best_similarities[l, 0]:
                        if l < k - 1:
                            # Shift the later values down one position
                            best_similarities[l+1] = best_similarities[l]
                        best_similarities[l] = [similarity, j]
                    else:
                        break

        # Store the indices of the K nearest neighbors
        knn_matrix[i] = best_similarities[:, 1].astype(int)
        similarities_matrix[i] = best_similarities[:, 0].astype(float)
        # Reset the best similarities for the next user
        best_similarities.fill(np.inf)

        end_time = time.time()  # End timing for the current user
        if(print_ and i % print_every_n == 0): print(f"Processed user {i+1}/{num_users} in {end_time - start_time:.2f} seconds.")

    end_total_time = time.time()  # End timing for the whole function
    if(print_): print(f"Total time taken: {end_total_time - start_total_time:.2f} seconds.")


    return knn_matrix, similarities_matrix


Okey the first thing I want to test out is how long the training for different k values takes

In [25]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_0_fill, 3, 'euclidean')

Processed user 1/671 in 0.06 seconds.
Processed user 101/671 in 0.06 seconds.
Processed user 201/671 in 0.06 seconds.
Processed user 301/671 in 0.06 seconds.
Processed user 401/671 in 0.06 seconds.
Processed user 501/671 in 0.06 seconds.
Processed user 601/671 in 0.06 seconds.
Total time taken: 39.33 seconds.


## **4. Rule of Thumb:**
Some $K = \sqrt{n}$, where $n$ is the total number of samples in your training dataset. This formul can give you a starting point, but it's still essential to experiment around this value to find the optimal $K$.

In [26]:
train_data_matrix_avg_fill.shape

(671, 9066)

In [27]:
import math
Ksqrt = math.sqrt(train_data_matrix_avg_fill.shape[0])
Ksqrt

25.903667693977237

for now we're just gonna test out the time taken to compute the largest k's we are gonna use in crossvalidation to see if we need to implkament parlaaization in the case the algorithem takes too long

In [28]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_0_fill, int(2 * Ksqrt), 'euclidean')

Processed user 1/671 in 0.07 seconds.
Processed user 101/671 in 0.07 seconds.
Processed user 201/671 in 0.06 seconds.
Processed user 301/671 in 0.06 seconds.
Processed user 401/671 in 0.07 seconds.
Processed user 501/671 in 0.07 seconds.
Processed user 601/671 in 0.07 seconds.
Total time taken: 44.51 seconds.


In [29]:
knn_matrix.shape

(671, 51)

It really didn't take so long... so I don't think I'm gonna go after paralarization just yet.

## Implamenting Prediction and Evaluation

In [30]:
knn_matrix.shape # 671 user and its 51 neighbors

(671, 51)

In [31]:
train_data_matrix_0_fill.shape # 671 users and 9066 movies

(671, 9066)

In [32]:
train_data_matrix_0_fill

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [33]:
set(train_data[['item_id']].sort_values(by=['item_id'])['item_id'].astype(int).to_list())

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 32797,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 65585,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 65601,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 32840,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 32853,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 65638,
 107,
 108,
 65642,
 110,
 111,
 112,
 113,
 114,
 32882,
 116,
 117,
 118,
 119,
 121,
 122,
 123,
 124,
 125,
 126,
 32892,
 129,
 130,
 131,
 132,
 32898,
 65665,
 135,
 137,
 98441,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 32943,
 183,
 184,
 185,
 186,
 187,
 

Below is the code for the function that predicts movie ratings for users

In [34]:
def predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix=True):
    # Extract the K nearest neighbors for the given user
    user_index = int(user_index)
    prev_index = 0
    movie_index = int(movie_index)
    neighbors_indices = knn_matrix[user_index]

    # Collect ratings for the target movie given by the neighbors
    if movie_index > ratings_matrix.shape[1]:
       movie_index = prev_index + 1

    neighbor_ratings = ratings_matrix[neighbors_indices, movie_index]

    # Also, collect the similarities for weighting, if applicable
    neighbor_similarities = similarities_matrix[user_index]

    # Filter out zeros (assuming they represent missing ratings)
    valid_indices = neighbor_ratings != 0
    valid_ratings = neighbor_ratings[valid_indices]
    valid_similarities = neighbor_similarities[valid_indices]

    prev_index = movie_index

    if use_similarities_matrix and valid_ratings.size > 0:
        # Compute weighted average of valid ratings
        weighted_sum = np.sum(valid_ratings * valid_similarities)
        sum_of_weights = np.sum(valid_similarities)
        predicted_rating = weighted_sum / sum_of_weights if sum_of_weights > 0 else np.nan
    elif valid_ratings.size > 0:
        # Compute simple average of valid ratings
        predicted_rating = valid_ratings.mean()
    else:
        # If there are no valid ratings, use the global average rating
        global_average = np.nanmean(ratings_matrix[ratings_matrix != 0])
        predicted_rating = global_average

    return predicted_rating


Below are a bunch of outputs for insights into how our matrecies and data are structured after training

In [35]:
knn_matrix[0, :]

array([546, 563,  72,  14, 623,  29, 451, 379, 293, 471, 467, 387, 231,
       101, 508,  22, 653, 579, 517, 310, 211, 118, 586,  55, 480, 241,
       305, 528, 663,  47, 357, 212, 298, 574, 595, 479, 606, 460, 198,
       104,  18, 474, 404, 406,  16, 462, 267, 261, 576, 543, 584])

In [36]:
train_data_matrix_0_fill[knn_matrix[0], 0]

array([3.5, 4. , 5. , 2. , 5. , 4. , 3.5, 4. , 4. , 5. , 4. , 0. , 3. ,
       0. , 3. , 3. , 5. , 4. , 5. , 3. , 3. , 2. , 0. , 4. , 4. , 5. ,
       3. , 4. , 3.5, 4. , 5. , 3. , 0. , 3. , 3.5, 5. , 4.5, 3.5, 0. ,
       0. , 3. , 4.5, 0. , 4. , 0. , 3. , 5. , 2.5, 5. , 0. , 4. ])

In [37]:
train_data

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
77406,69,344,4.0,1366831253
77407,452,4874,3.0,1044681111
77408,575,1201,5.0,1012594455
77409,17,1212,5.0,1127468800


In [38]:
train_data[(train_data.user_id == 1) & (train_data.item_id == 31)]

,user_id,item_id,rating,timestamp
0,1,31,2.5,1260759144


In [39]:
len(train_data)

77411

In [40]:
for index, row in train_data.iterrows():
        user_index = row['user_id']
        movie_index1=row['item_id']
        print(movie_index1)

Streaming output truncated to the last 5000 lines.
539.0
3082.0
356.0
51255.0
780.0
986.0
648.0
4321.0
136598.0
2639.0
9.0
300.0
474.0
2793.0
595.0
5378.0
11.0
8636.0
7149.0
6303.0
8910.0
1035.0
4878.0
4018.0
858.0
1193.0
2080.0
1233.0
1027.0
3753.0
2689.0
2468.0
1036.0
1097.0
3048.0
1198.0
3741.0
1610.0
1248.0
6953.0
4262.0
3478.0
33794.0
7.0
2423.0
1284.0
903.0
1269.0
527.0
1199.0
611.0
348.0
1862.0
2334.0
47.0
1595.0
593.0
4357.0
1198.0
235.0
54997.0
231.0
912.0
103384.0
65982.0
4239.0
2565.0
1480.0
3468.0
1588.0
1449.0
30793.0
2826.0
1810.0
1923.0
4992.0
356.0
1356.0
2121.0
4878.0
1260.0
2539.0
5507.0
1267.0
3897.0
344.0
4021.0
296.0
2716.0
1278.0
648.0
778.0
8961.0
6062.0
592.0
924.0
1721.0
1246.0
62155.0
122892.0
2622.0
1196.0
53972.0
7028.0
16.0
2716.0
4034.0
480.0
1193.0
5956.0
2390.0
502.0
648.0
5944.0
1250.0
1732.0
53125.0
4878.0
4041.0
1704.0
216.0
253.0
3617.0
608.0
1238.0
3869.0
6539.0
16.0
4678.0
329.0
2115.0
750.0
1721.0
26776.0
2023.0
3751.0
3072.0
1665.0
193.0
344.0
50

In [41]:
for index, row in train_data.iterrows():
    print(row['rating'])

Streaming output truncated to the last 5000 lines.
4.0
1.0
5.0
3.5
2.0
3.0
2.0
2.5
3.5
4.0
3.0
5.0
3.0
4.0
3.0
3.0
4.0
3.0
4.0
3.0
3.0
3.5
3.5
3.0
4.0
4.0
3.5
5.0
2.0
2.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
5.0
4.0
4.0
4.0
3.0
3.0
3.0
4.0
3.5
5.0
2.0
4.5
5.0
2.0
4.0
2.0
3.0
2.5
1.0
5.0
3.5
4.5
4.5
4.5
3.0
4.0
1.0
2.5
4.5
3.0
4.0
5.0
1.0
4.0
3.0
4.0
3.0
3.5
3.5
5.0
5.0
5.0
4.0
4.0
2.0
3.0
4.0
5.0
1.5
4.0
3.0
4.0
4.0
3.5
4.5
5.0
3.5
3.0
5.0
4.0
4.0
2.0
3.5
3.0
4.0
5.0
4.0
3.5
4.0
5.0
4.0
5.0
4.0
4.0
3.0
4.0
3.5
4.5
4.0
5.0
3.0
3.5
5.0
3.0
3.0
3.0
5.0
4.0
5.0
4.0
5.0
4.0
3.0
2.5
4.0
0.5
3.5
4.0
3.0
5.0
3.0
2.0
4.0
3.5
4.5
1.5
3.0
4.0
4.0
4.5
4.0
4.5
5.0
5.0
3.0
2.5
1.5
3.5
3.5
4.5
3.0
3.0
2.0
5.0
5.0
3.0
4.5
4.0
4.5
3.5
4.5
4.5
4.0
5.0
2.5
3.0
3.0
3.0
4.0
3.5
4.0
3.0
3.0
4.0
3.5
3.5
3.0
4.0
4.5
4.0
3.0
4.0
5.0
5.0
3.0
4.0
3.0
5.0
2.5
5.0
3.0
4.0
4.0
3.0
4.0
4.0
3.0
4.0
4.0
3.0
3.0
3.0
3.0
5.0
3.5
1.0
3.0
5.0
4.0
3.5
2.0
3.0
4.5
3.0
4.0
4.5
5.0
4.5
3.0
3.0
4.0
5.0
4.5
4.5
1.0
1.5
3.0
2.0
5.0
3.5
2

Below is the rather time expensive evaluate function that evaluates the perforamnce of the current knn model on a givven dataset. We will parallize this function shortly to make it run faster in some upcomming tests

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

def evaluate_model(knn_matrix, similarities_matrix, ratings_matrix, dataset, use_similarities_matrix = True, print_ = True, print_every_n = 500):

    predictions = []
    actual_ratings = []

    # Iterate through each row in the dataset to make predictions
    i=0
    for index, row in dataset.iterrows():
        user_index = int(row['user_id'])-1
        movie_index = int(row['item_id'])-1
        actual_rating = row['rating']
        # Adjust indices for zero-based indexing
        if not np.isnan(actual_rating):

            predicted_rating = predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix)
            # print("evaluating on value", actual_rating)
            # print("predictet value", predicted_rating)

            predictions.append(predicted_rating)
            actual_ratings.append(actual_rating)
        else:
            # print("value should be nan: ", col)
            pass
        i+=1
        if(print_ and i%print_every_n==0): print(f"done with row index: {i}, out of: {len(dataset)}")

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(actual_ratings, predictions))
    mae = mean_absolute_error(actual_ratings, predictions)

    if print_:
        print(f"RMSE: {rmse}")
        print(f"MAE: {mae}")

    return rmse, mae


Testing the predict function

In [43]:
predict_rating(0, 0, knn_matrix, similarities_matrix, train_data_matrix_0_fill)

3.8332888196358414

As we can see from above our knn predicted a rating of 3.83 for the user 0 and movie 0 and below we have some more insights into our data

In [44]:
dataset_array = train_data[['user_id', 'item_id', 'rating']].to_numpy()
dataset_array.shape

(77411, 3)

In [45]:
print(train_data_matrix_0_fill.shape)
print(knn_matrix.shape)

(671, 9066)
(671, 51)


### Now let's finally try evaluating our knn model

In [46]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, train_data)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, test_data)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

================================

Training RMSE: 1.3108213065825864

Training MAE: 1.0372405987684707

================================

Test RMSE: 1.303092362800314

Test MAE: 1.0299218571321986

================================

These results aren't half bad for the little training it took compared to SVD. Interestingly it's also worth noting how similar the training and test RMSE errors are which is either an indication that I did something wrong in my knn implamentationor it shows how prone to overfiting on training data other algorithems like neural network based aproaches and or SVD are!

Now lets see what happens if we don't use the use_similarities_matrix = True and set it to False. These should be a more standard "and simple" KNN implamentation.

In [47]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, train_data, False)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_0_fill, test_data, False)

print('=' * 40)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

========================================

Training RMSE: 1.3117922938674418

Training MAE: 1.038320313088493

========================================

Test RMSE: 1.3041755659637155

Test MAE: 1.0309168630279977

========================================

A 0.0002 difference in RMSE! I guess it's noteable if you're operating on billions! I'm kidding it might just be something wrong with my implamentation or my k value or the structure of the 100k movies dataset.

Next I want to try what happens if we use the initialization of avrage values for missing values instead of a 0.

*I know we prepared the matrix before and I even explicitly mentioned it but let's do it again for the sake of cohearence*

In [50]:
# Create the pivot table
train_data_matrix = train_data.pivot_table(index='user_id', columns='item_id', values='rating')
# Fill missing values with user averages
train_data_matrix_avg_fill = train_data_matrix.apply(lambda x: x.fillna(user_rating_avg[x.name]), axis=1)
display(train_data_matrix_avg_fill.head())
train_data_matrix_avg_fill = train_data_matrix_avg_fill.to_numpy()
# Display the shape and first few rows of the matrix
print(train_data_matrix_avg_fill.shape)

item_id,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,...,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000,2.550000
2,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,4.000000,...,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333,3.493333
3,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,...,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000,3.590000
4,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.000000,...,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259,4.348259
5,3.917526,3.917526,4.000000,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,...,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526,3.917526


(671, 9066)


In [51]:
knn_matrix, similarities_matrix = k_nearest_neighbors(train_data_matrix_avg_fill, int(2 * Ksqrt), 'euclidean')

Processed user 1/671 in 0.03 seconds.
Processed user 101/671 in 0.03 seconds.
Processed user 201/671 in 0.03 seconds.
Processed user 301/671 in 0.03 seconds.
Processed user 401/671 in 0.03 seconds.
Processed user 501/671 in 0.03 seconds.
Processed user 601/671 in 0.03 seconds.
Total time taken: 18.87 seconds.


In [52]:
train_rmse, train_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_avg_fill, train_data)
test_rmse, test_mae = evaluate_model(knn_matrix, similarities_matrix, train_data_matrix_avg_fill, test_data)

print('KNN fill avg')
print('=' * 40)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print('=' * 40)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")

done with row index: 500, out of: 77411
done with row index: 1000, out of: 77411
done with row index: 1500, out of: 77411
done with row index: 2000, out of: 77411
done with row index: 2500, out of: 77411
done with row index: 3000, out of: 77411
done with row index: 3500, out of: 77411
done with row index: 4000, out of: 77411
done with row index: 4500, out of: 77411
done with row index: 5000, out of: 77411
done with row index: 5500, out of: 77411
done with row index: 6000, out of: 77411
done with row index: 6500, out of: 77411
done with row index: 7000, out of: 77411
done with row index: 7500, out of: 77411
done with row index: 8000, out of: 77411
done with row index: 8500, out of: 77411
done with row index: 9000, out of: 77411
done with row index: 9500, out of: 77411
done with row index: 10000, out of: 77411
done with row index: 10500, out of: 77411
done with row index: 11000, out of: 77411
done with row index: 11500, out of: 77411
done with row index: 12000, out of: 77411
done with ro

Wow okey! I was not expacting a bigger error by 0.4. My theory is  that an avrage initialization makes the data steer towards the avrage for no real practical reason and thereby maybe it takes different nearest neighbours. Or you know.. my implamentation is wrong.

## Testing different K values

In [53]:

from concurrent.futures import ThreadPoolExecutor
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
import time


Bellow is the parallized code you don't need to put much thouight to it it's there mainly for speeding up my testing times.

In [58]:


def evaluate_model_concurrent(knn_matrix, similarities_matrix, ratings_matrix, dataset, use_similarities_matrix=True, workers=4):
    def worker(row):
        user_index, movie_index, actual_rating = row['user_id'].astype(int)-1, row['item_id'].astype(int)-1, row['rating']
        if not np.isnan(actual_rating):
            predicted_rating = predict_rating(user_index, movie_index, knn_matrix, similarities_matrix, ratings_matrix, use_similarities_matrix)
            return predicted_rating, actual_rating

    with ThreadPoolExecutor(max_workers=workers) as executor:
        results = executor.map(worker, [row for i, row in dataset.iterrows()])

    predictions, actual_ratings = zip(*[r for r in results if r is not None])

    # Calculate RMSE
    rmse = sqrt(mean_squared_error(actual_ratings, predictions))

    return rmse


In [55]:
import os
import multiprocessing

default_workers = os.cpu_count()
default_workers


12

Now let's get to the juicy part. Again like with the SVD testing let's write a function that tests different k values.
In this function we don't calculate a new knn model for each k because the first k-1 values in any knn row are the same as the the whole k-1 row

In [57]:
def test_different_ks(start_k, end_k, step, dataset_matrix, similarity_type='euclidean', use_similarities_matrix=True, train_data_portion=0.5, test_data_portion =1):
    train_rmse_values = []
    test_rmse_values = []

    cur_train_data = train_data[:int(len(train_data)*train_data_portion)]
    cur_test_data = train_data[:int(len(test_data)*test_data_portion)]

    # Calculate KNN and similarities matrices for the largest k value
    print(f"Calculating KNN and similarities matrices for k={end_k}")
    full_knn_matrix, full_similarities_matrix = k_nearest_neighbors(dataset_matrix, end_k, similarity_type)

    for k in range(start_k, end_k + 1, step):
        print(f"Testing with k={k}")
        start_time = time.time()

        # Use only the first k columns of the precomputed matrices
        cur_knn_matrix = full_knn_matrix[:, :k]
        cur_similarities_matrix = full_similarities_matrix[:, :k]

        train_rmse = evaluate_model(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_train_data, use_similarities_matrix)
        test_rmse = evaluate_model(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_test_data, use_similarities_matrix)

        train_rmse_values.append(train_rmse)
        test_rmse_values.append(test_rmse)
        end_time = time.time()
        print(f"k = {k}, train_rmse = {train_rmse}, test_rmse = {test_rmse}, time taken = {end_time - start_time:.2f} seconds")

    return full_knn_matrix, full_similarities_matrix, train_rmse_values, test_rmse_values


the parallel version of the above code that just calls the parallel evaluate function

In [59]:
def test_different_ks_parallel(start_k, end_k, step, dataset_matrix, similarity_type='euclidean', use_similarities_matrix=True, train_data_portion=0.5, test_data_portion =1, full_knn_matrix = None, full_similarities_matrix = None):
    cur_train_data = train_data[:int(len(train_data)*train_data_portion)]
    cur_test_data = train_data[:int(len(test_data)*test_data_portion)]
    train_rmse_values = []
    test_rmse_values = []

    #if(full_knn_matrix == None or full_similarities_matrix == None):
       # print(f"Calculating KNN and similarities matrices for k={end_k}")
        #full_knn_matrix, full_similarities_matrix = k_nearest_neighbors(dataset_matrix, end_k, similarity_type)

    for k in range(start_k, end_k + 1, step):
        print(f"Testing with k={k}")
        start_time = time.time()
        # Use only the first k columns of the precomputed matrices
        cur_knn_matrix = full_knn_matrix[:, :k]
        cur_similarities_matrix = full_similarities_matrix[:, :k]

        train_rmse = evaluate_model_concurrent(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_train_data, use_similarities_matrix,workers=16)
        test_rmse = evaluate_model_concurrent(cur_knn_matrix, cur_similarities_matrix, dataset_matrix, cur_test_data, use_similarities_matrix,workers=16)

        train_rmse_values.append(train_rmse)
        test_rmse_values.append(test_rmse)
        end_time = time.time()
        print(f"k = {k}, train_rmse = {train_rmse}, test_rmse = {test_rmse}")
        print(f"time taken to test k={k} - {end_time - start_time:.2f} seconds")

    return full_knn_matrix, full_similarities_matrix, train_rmse_values, test_rmse_values

In [60]:
int(4 * Ksqrt)

103

In [ ]:
knn_matrix, similarities_matrix, train_rmse_values, test_rmse_values = test_different_ks(
    start_k=1,
    end_k=200,
    step=1,
    dataset_matrix=train_data_matrix_0_fill,
    similarity_type='euclidean',
    use_similarities_matrix=True
)

Calculating KNN and similarities matrices for k=200
Processed user 1/671 in 0.13 seconds.
Processed user 101/671 in 0.13 seconds.
Processed user 201/671 in 0.13 seconds.
Processed user 301/671 in 0.13 seconds.
Processed user 401/671 in 0.13 seconds.
Processed user 501/671 in 0.13 seconds.
Processed user 601/671 in 0.12 seconds.
Total time taken: 86.07 seconds.
Testing with k=1
done with row index: 500, out of: 38705
done with row index: 1000, out of: 38705
done with row index: 1500, out of: 38705
done with row index: 2000, out of: 38705
done with row index: 2500, out of: 38705
done with row index: 3000, out of: 38705
done with row index: 3500, out of: 38705
done with row index: 4000, out of: 38705
done with row index: 4500, out of: 38705
done with row index: 5000, out of: 38705
done with row index: 5500, out of: 38705
done with row index: 6000, out of: 38705
done with row index: 6500, out of: 38705
done with row index: 7000, out of: 38705
done with row index: 7500, out of: 38705
done w

In [ ]:
ks = range(1, int(4 * Ksqrt)+1, 1)

plt.plot(ks, train_rmse_values, label='Train RMSE', marker='o')
plt.plot(ks, test_rmse_values, label='Test RMSE', marker='s')
plt.xlabel('k')
plt.ylabel('RMSE')
plt.title('RMSE for Different Values of k')
plt.legend()
plt.grid(True)
plt.show()